In [135]:
import quandl
import pandas as pd
import numpy as np
from sklearn.svm import SVR # Máquina soporte vectorial 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split,RandomizedSearchCV #Hiperparametrización
quandl.ApiConfig.api_key = 'DEN8cmx6MYrtzyeUdDeh' #????

In [136]:
%%time
tickers = ['FB','GOOGL','AAPl','AMZN','TSLA','MSFT','AMD','INTC','T','NFLX']
Validate = [] #Conjunto para validación 
Train = [] #Conjunto de entrenamiento

for ticker in tickers:
    df = quandl.get("WIKI/"+ticker)
    #print (df)
    df = df.iloc[-500:]
    func = ['mean','std','skew','kurt'] #Estadísticos de interés
    time = [2,4,7,30,60,120,250] #Días tomados artibatriamente  
    Z = pd.DataFrame()  
    for f in func: 
        for t in time:
            aux = pd.DataFrame()
            for i in range(249):
                aux[ df.index[i+251] ] = list( df.iloc[i-t+251:i+251]['Adj. Close'] ) #
            Z[f+str(t)] = aux.apply(f,axis=0)  
            

    Z.drop(['skew2','kurt2'],axis=1, inplace =True)
    W = pd.merge(Z, df[['Adj. Close']],left_index=True,right_index=True)    
    X = W.drop(['Adj. Close'],1).fillna(0)
    mm1 = MinMaxScaler()
    mm2 = MinMaxScaler()
    mm1.fit(X)
    mm2.fit(y)
    Xm = pd.DataFrame(mm1.transform(X))
    ym = pd.DataFrame(mm2.transform(y))
    Xt, Xv, yt, yv = train_test_split(Xm, ym, test_size = 0.3)
    model =SVR()
    model.fit(Xt,yt)
    # Hiperparametrizando
    param_grid = dict( C = np.arange(0.1,1,0.1),
                  epsilon=np.arange(0.1,1,0.1),
                  kernel = ['linear', 'poly', 'rbf', 'sigmoid'],
                  degree = range(1,6),
                  max_iter= [500])
    grid = GridSearchCV(cv=3,
                  verbose=True,
                  estimator=model,
                  n_jobs=-1,
                  param_grid=param_grid)
    grid.fit(Xm,ym)
    model = grid.best_estimator_
    model.fit(Xt,yt)
    print ticker
    print 'ACC Validate {:.2%}'.format(model.score(Xv,yv))
    print 'ACC Train    {:.2%}'.format(model.score(Xt,yt))
    Validate.append(model.score(Xv,yv))
    Train.append(model.score(Xt,yt))
print '\n'
print 'Promedio Validate  {:.2%}'.format(np.array(Validate).mean())
print 'Promedio Train     {:.2%}'.format(np.array(Train).mean())

Fitting 3 folds for each of 1620 candidates, totalling 4860 fits


[Parallel(n_jobs=-1)]: Done 436 tasks      | elapsed:    6.0s
/home/ciprian/.local/lib/python2.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done 2236 tasks      | elapsed:   24.2s
/home/ciprian/.local/lib/python2.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done 4860 out of 4860 | elapsed:   50.5s finished


FB
ACC Validate 83.17%
ACC Train    81.00%
Fitting 3 folds for each of 1620 candidates, totalling 4860 fits


[Parallel(n_jobs=-1)]: Done 324 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1524 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 3524 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done 4860 out of 4860 | elapsed:   50.6s finished


GOOGL
ACC Validate 91.45%
ACC Train    90.66%
Fitting 3 folds for each of 1620 candidates, totalling 4860 fits


...

[Parallel(n_jobs=-1)]: Done 244 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 1444 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done 4388 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 4860 out of 4860 | elapsed:   58.7s finished


T
ACC Validate 26.48%
ACC Train    25.03%
Fitting 3 folds for each of 1620 candidates, totalling 4860 fits


[Parallel(n_jobs=-1)]: Done 340 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 1540 tasks      | elapsed:   24.1s


NFLX
ACC Validate 96.97%
ACC Train    95.80%


Promedio Validate  69.91%
Promedio Train     69.73%
CPU times: user 6min 6s, sys: 10.7 s, total: 6min 17s
Wall time: 12min 22s


[Parallel(n_jobs=-1)]: Done 4860 out of 4860 | elapsed:   48.3s finished


Observación: Los datos que estaban dando como Target son continuos entonces se debía cambiar la SVM en modo regresión 